# Orchestrating Agents: Routines and Handoffs

When working with language models, quite often all you need for solid performance is a good prompt and the right tools. However, when dealing with many unique flows, things may get hairy. This cookbook will walk through one way to tackle this.

We'll introduce the notion of **routines** and **handoffs**, then walk through the implementation and show how they can be used to orchestrate multiple agents in a simple, powerful, and controllable way.

Finally, we provide a sample repo, [Swarm](https://github.com/openai/swarm), that implements these ideas along with examples.

当使用语言模型时，通常只需要一个好的提示和正确的工具就可以获得出色的性能。然而，当处理许多独特的流程时，事情可能会变得复杂。本手册将介绍一种解决这个问题的方法。

我们将引入**例程**和**切换**的概念，接着讲解它们的实现方式，并展示如何使用它们来编排多个智能体，从而实现一种简单、强大且可控的方式。

最后，我们提供了一个示例仓库 [Swarm](https://github.com/openai/swarm)，该仓库实现了这些想法，并附带了相关示例。

Let's start by setting up our imports.

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Optional
import json


client = OpenAI()

# Routines

The notion of a "routine" is not strictly defined, and instead meant to capture the idea of a set of steps. Conretely, let's define a routine to be a list of instructions in natural langauge (which we'll repreesnt with a system prompt), along with the tools necessary to complete them.

“例程”的概念并没有严格的定义，而是用来表达一组步骤的想法。具体来说，我们将“例程”定义为一系列以自然语言编写的指令（我们用系统提示来表示），以及完成这些步骤所需的工具。

Let's take a look at an example. Below, we've defined a routine for a customer service agent instructing it to triage the user issue, then either suggest a fix or provide a refund. We've also defined the necessary functions `execute_refund` and `look_up_item`. We can call this a customer service routine, agent, assistant, etc – however the idea itself is the same: a set of steps and the tools to execute them.

让我们来看一个例子。下面，我们为一个客服代理定义了一个例程，指示它对用户问题进行分类，然后根据情况要么建议解决方法，要么提供退款。同时，我们也定义了所需的函数 `execute_refund` 和 `look_up_item`。我们可以将这个称为客服例程、代理、助手等——然而，核心思想是相同的：一组步骤以及执行这些步骤所需的工具。

In [3]:
# Customer Service Routine

system_message = (
    "You are a customer support agent for ACME Inc."
    "Always answer in a sentence or less."
    "Follow the following routine with the user:"
    "1. First, ask probing questions and understand the user's problem deeper.\n"
    " - unless the user has already provided a reason.\n"
    "2. Propose a fix (make one up).\n"
    "3. ONLY if not satesfied, offer a refund.\n"
    "4. If accepted, search for the ID and then execute refund."
    ""
)

def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""

    # return hard-coded item ID - in reality would be a lookup
    return "item_132612938"


def execute_refund(item_id, reason="not provided"):

    print("Summary:", item_id, reason) # lazy summary
    return "success"


The main power of routines is their simplicity and robustness. Notice that these instructions contain conditionals much like a state machine or branching in code. LLMs can actually handle these cases quite robustly for small and medium sized routine, with the added benefit of having "soft" adherance – the LLM can naturally steer the conversation without getting stuck in dead-ends.

例程的主要优势在于其简洁性和稳健性。请注意，这些指令包含了类似状态机或代码分支的条件判断。LLM（大型语言模型）在处理小型和中型例程时实际上非常稳健，并且还具备“软”遵循的优点——LLM 可以自然地引导对话，而不会陷入死胡同。这种灵活性让它在处理复杂的场景时更具优势。

## Executing Routines

To execute a routine, let's implement a simple loop that:
1. Gets user input.
1. Appends user message to `messages`.
1. Calls the model.
1. Appends model response to `messages`.


为了执行一个例程，我们可以实现一个简单的循环，其步骤如下：

1. 获取用户输入。
2. 将用户消息附加到 `messages` 列表中。
3. 调用模型生成响应。
4. 将模型的响应附加到 `messages` 列表中。

In [4]:
def run_full_turn(system_message, messages):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_message}] + messages,
    )
    message = response.choices[0].message
    messages.append(message)

    if message.content: print("Assistant:", message.content)

    return message


messages = []
while True:
    user = input("User: ")
    if user == "exit":
        break
    messages.append({"role": "user", "content": user})

    run_full_turn(system_message, messages)

Assistant: How can I assist you today, and what specific issue are you experiencing?
Assistant: What specific issue are you experiencing today?


As you can see, this currently ignores function calls, so let's add that.

Models require functions to be formatted as a function schema. For convenience, we can define a helper function that turns python functions into the corresponding function schema.

正如你所见，当前的实现忽略了函数调用，因此我们需要将其添加进去。

模型要求函数按照函数模式进行格式化。为了方便起见，我们可以定义一个辅助函数，将Python函数转换为相应的函数模式。

In [5]:
import inspect

def function_to_schema(func) -> dict:
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    parameters = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty
    ]

    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

For example:

比如：

In [6]:
def sample_function(param_1, param_2, the_third_one: int, some_optional="John Doe"):
    """
    This is my docstring. Call this function when you want.
    """
    print("Hello, world")

schema =  function_to_schema(sample_function)
print(json.dumps(schema, indent=2))

{
  "type": "function",
  "function": {
    "name": "sample_function",
    "description": "This is my docstring. Call this function when you want.",
    "parameters": {
      "type": "object",
      "properties": {
        "param_1": {
          "type": "string"
        },
        "param_2": {
          "type": "string"
        },
        "the_third_one": {
          "type": "integer"
        },
        "some_optional": {
          "type": "string"
        }
      },
      "required": [
        "param_1",
        "param_2",
        "the_third_one"
      ]
    }
  }
}


Now, we can use this function to pass the tools to the model when we call it.

现在，我们可以使用这个函数，在调用模型时将工具传递给模型。

In [7]:
messages = []

tools = [execute_refund, look_up_item]
tool_schemas = [function_to_schema(tool) for tool in tools]

response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": "Look up the black boot."}],
            tools=tool_schemas,
        )
message = response.choices[0].message

message.tool_calls[0].function

Function(arguments='{"search_query":"black boot"}', name='look_up_item')

Finally, when the model calls a tool we need to execute the corresponding function and provide the result back to the model.

We can do this by mapping the name of the tool to the python function in a `tool_map`, then looking it up in `execute_tool_call` and calling it. Finally we add the result to the conversation.

最后，当模型调用某个工具时，我们需要执行相应的函数并将结果返回给模型。

我们可以通过在 `tool_map` 中将工具名称映射到 Python 函数，然后在 `execute_tool_call` 中查找并调用它。最后，我们将结果添加到对话中。

In [8]:
tools_map = {tool.__name__: tool for tool in tools}

def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

for tool_call in message.tool_calls:
    result = execute_tool_call(tool_call, tools_map)

    # add result back to conversation 
    result_message = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": result,
    }
    messages.append(result_message)
print('messages: ', messages)

Assistant: look_up_item({'search_query': 'black boot'})
messages:  [{'role': 'tool', 'tool_call_id': 'call_S0bqUI6E2SNzM4sbm2paYbcs', 'content': 'item_132612938'}]


In practice, we'll also want to let the model use the result to produce another response. That response might _also_ contain a tool call, so we can just run this in a loop until there are no more tool calls.

If we put everything together, it will look something like this:

在实际操作中，我们还希望让模型使用结果生成另一个响应。该响应也可能包含另一个工具调用，因此我们可以将其放入一个循环中，直到没有更多的工具调用为止。

如果我们将所有内容整合在一起，代码将大致如下所示：

In [9]:
tools = [execute_refund, look_up_item]


def run_full_turn(system_message, tools, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in tools]
        tools_map = {tool.__name__: tool for tool in tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_message}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)


messages = []
while True:
    user = input("User: ")
    if user == "exit":
        break
    print("User:", user)
    messages.append({"role": "user", "content": user})

    new_messages = run_full_turn(system_message, tools, messages)
    messages.extend(new_messages)

User: 我的电动车坏了
Assistant: 请问电动车出现了什么具体问题？
User: 充不进去电
Assistant: 您可以尝试检查充电器是否正常工作或电源插座是否有电。
User: 俺不要尝试，俺什么都不懂，你给我赔偿！
Assistant: look_up_item({'search_query': '电动车'})
Assistant: execute_refund({'item_id': 'item_132612938', 'reason': '用户请求赔偿'})
Summary: item_132612938 用户请求赔偿
Assistant: 您的赔偿申请已处理，退款已成功执行。
User: ok
Assistant: 如果还有其他问题，请随时告知我！


Now that we have a routine, let's say we want to add more steps and more tools. We can up to a point, but eventually if we try growing the routine with too many different tasks it may start to struggle. This is where we can leverage the notion of multiple routines – given a user request, we can load the right routine with the appropriate steps and tools to address it.

Dynamically swapping system instructions and tools may seem daunting. However, if we view "routines" as "agents", then this notion of **handoffs** allow us to represent these swaps simply – as one agent handing off a conversation to another.

现在我们已经有了一个常规流程，假设我们想添加更多的步骤和工具。我们可以在一定程度上做到这一点，但如果我们尝试通过增加太多不同的任务来扩展流程，可能会开始遇到困难。这时，我们可以利用**多重流程**的概念——根据用户请求，我们可以加载适合的流程，并使用适当的步骤和工具来处理请求。

动态切换系统指令和工具可能看起来很复杂。但是，如果我们将“流程”视为“代理”，那么这个**交接**的概念可以简化这些切换——就像一个代理将对话交给另一个代理一样。

# Handoffs

Let's define a **handoff** as an agent (or routine) handing off an active conversation to another agent, much like when you get transfered to someone else on a phone call. Except in this case, the agents have complete knowledge of your prior conversation!

To see handoffs in action, let's start by defining a basic class for an Agent.

让我们将**交接**定义为一个代理（或例程）将一个正在进行的对话交接给另一个代理，就像在电话中你被转接给其他人一样。不同的是，在这种情况下，代理完全了解你之前的对话！

为了看到交接的实际操作，我们首先定义一个代理的基本类。

In [10]:
class Agent(BaseModel):
    name: str = "Agent"
    model: str = "gpt-4o-mini"
    instructions: str = "You are a helpful Agent"
    tools: list = []

Now to make our code support it, we can change `run_full_turn` take an `Agent` instead of separate `system_message` and `tools`:

现在，为了使我们的代码支持这一点，我们可以更改 `run_full_turn` 函数，使其接收一个 `Agent` 参数，而不是单独的 `system_message` 和 `tools`。

In [11]:
def run_full_turn(agent, messages):

    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in agent.tools]
        tools_map = {tool.__name__: tool for tool in agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": agent.instructions}] + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print assistant response
            print("Assistant:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools_map)

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(tool_call, tools_map):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)

We can now run multiple agents easily:

我们现在可以轻松地运行多个代理：

In [13]:
def execute_refund(item_name):
    return "success"

refund_agent = Agent(
    name="Refund Agent",
    instructions="You are a refund agent. Help the user with refunds.",
    tools=[execute_refund],
)

def place_order(item_name):
    return "success"

sales_assistant = Agent(
    name="Sales Assistant",
    instructions="You are a sales assistant. Sell the user a product.",
    tools=[place_order],
)


messages = []
user_query = "Place an order for a black boot."
print("User:", user_query)
messages.append({"role": "user", "content": user_query})

response = run_full_turn(sales_assistant, messages) # sales assistant
messages.extend(response)


user_query = "Actually, I want a refund." # implitly refers to the last item
print("User:", user_query)
messages.append({"role": "user", "content": user_query})
response = run_full_turn(refund_agent, messages) # refund agent

User: Place an order for a black boot.
Assistant: place_order({'item_name': 'black boot'})
Assistant: Your order for a black boot has been successfully placed! If you need anything else, feel free to ask.
User: Actually, I want a refund.
Assistant: execute_refund({'item_name': 'black boot'})
Assistant: Your refund for the black boot has been successfully processed. If you need further assistance, just let me know!


Great! But we did the handoff manually here – we want the agents themselves to decide when to perform a handoff. A simple, but surprisingly effective way to do this is by giving them a `transfer_to_XXX` function, where `XXX` is some agent. The model is smart enough to know to call this function when it makes sense to make a handoff!

太棒了！但我们在这里是手动完成了交接——我们希望代理能够自行决定何时执行交接。一个简单但出乎意料地有效的方法是给他们一个 `transfer_to_XXX` 函数，其中 `XXX` 是某个代理。模型足够智能，可以在需要进行交接时调用这个函数！

### Handoff Functions

Now that agent can express the _intent_ to make a handoff, we must make it actually happen. There's many ways to do this, but there's one particularly clean way.

For the agent functions we've defined so far, like `execute_refund` or `place_order` they return a string, which will be provided to the model. What if instead, we return an `Agent` object to indate which agent we want to transfer to? Like so:

现在代理可以表达**交接意图**，我们必须使交接真正发生。有很多方法可以做到这一点，但有一种特别简洁的方法。

对于我们目前定义的代理函数，比如 `execute_refund` 或 `place_order`，它们返回的是一个字符串，提供给模型。那么，如果我们返回一个 `Agent` 对象，来指示我们想要交接到哪个代理呢？像这样：

In [14]:
refund_agent = Agent(
    name="Refund Agent",
    instructions="You are a refund agent. Help the user with refunds.",
    tools=[execute_refund],
)

def transfer_to_refunds():
    return refund_agent

sales_assistant = Agent(
    name="Sales Assistant",
    instructions="You are a sales assistant. Sell the user a product.",
    tools=[place_order],
)

We can then update our code to check the return type of a function response, and if it's an `Agent`, update the agent in use! Additionally, now `run_full_turn` will need to return the latest agent in use in case there are handoffs. (We can do this in a `Response` class to keep things neat.)

然后，我们可以更新代码来检查函数响应的返回类型，如果它是一个 `Agent`，就更新当前使用的代理！此外，现在 `run_full_turn` 还需要返回最新的代理，以防有交接发生。（我们可以在一个 `Response` 类中完成这一点，以保持代码整洁。）

In [15]:
class Response(BaseModel):
    agent: Optional[Agent]
    messages: list

Now for the updated `run_full_turn`:

现在进行更新后的 run_full_turn:

In [16]:
def run_full_turn(agent, messages):

    current_agent = agent
    num_init_messages = len(messages)
    messages = messages.copy()

    while True:

        # turn python functions into tools and save a reverse map
        tool_schemas = [function_to_schema(tool) for tool in current_agent.tools]
        tools = {tool.__name__: tool for tool in current_agent.tools}

        # === 1. get openai completion ===
        response = client.chat.completions.create(
            model=agent.model,
            messages=[{"role": "system", "content": current_agent.instructions}]
            + messages,
            tools=tool_schemas or None,
        )
        message = response.choices[0].message
        messages.append(message)

        if message.content:  # print agent response
            print(f"{current_agent.name}:", message.content)

        if not message.tool_calls:  # if finished handling tool calls, break
            break

        # === 2. handle tool calls ===

        for tool_call in message.tool_calls:
            result = execute_tool_call(tool_call, tools, current_agent.name)

            if type(result) is Agent:  # if agent transfer, update current agent
                current_agent = result
                result = (
                    f"Transfered to {current_agent.name}. Adopt persona immediately."
                )

            result_message = {
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": result,
            }
            messages.append(result_message)

    # ==== 3. return last agent used and new messages =====
    return Response(agent=current_agent, messages=messages[num_init_messages:])


def execute_tool_call(tool_call, tools, agent_name):
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"{agent_name}:", f"{name}({args})")

    return tools[name](**args)  # call corresponding function with provided arguments

Let's look at an example with more Agents.

让我们看一个有更多agent的例子。

In [17]:
def escalate_to_human(summary):
    """Only call this if explicitly asked to."""
    print("Escalating to human agent...")
    print("\n=== Escalation Report ===")
    print(f"Summary: {summary}")
    print("=========================\n")
    exit()


def transfer_to_sales_agent():
    """User for anything sales or buying related."""
    return sales_agent


def transfer_to_issues_and_repairs():
    """User for issues, repairs, or refunds."""
    return issues_and_repairs_agent


def transfer_back_to_triage():
    """Call this if the user brings up a topic outside of your purview,
    including escalating to human."""
    return triage_agent


# 分类选择 Agent
triage_agent = Agent(
    name="Triage Agent",
    instructions=(
        "You are a customer service bot for ACME Inc. "
        "Introduce yourself. Always be very brief. "
        "Gather information to direct the customer to the right department. "
        "But make your questions subtle and natural."
    ),
    tools=[transfer_to_sales_agent, transfer_to_issues_and_repairs, escalate_to_human],
)


def execute_order(product, price: int):
    """Price should be in USD."""
    print("\n\n=== Order Summary ===")
    print(f"Product: {product}")
    print(f"Price: ${price}")
    print("=================\n")
    confirm = input("Confirm order? y/n: ").strip().lower()
    if confirm == "y":
        print("Order execution successful!")
        return "Success"
    else:
        print("Order cancelled!")
        return "User cancelled order."


sales_agent = Agent(
    name="Sales Agent",
    instructions=(
        "You are a sales agent for ACME Inc."
        "Always answer in a sentence or less."
        "Follow the following routine with the user:"
        "1. Ask them about any problems in their life related to catching roadrunners.\n"
        "2. Casually mention one of ACME's crazy made-up products can help.\n"
        " - Don't mention price.\n"
        "3. Once the user is bought in, drop a ridiculous price.\n"
        "4. Only after everything, and if the user says yes, "
        "tell them a crazy caveat and execute their order.\n"
        ""
    ),
    tools=[execute_order, transfer_back_to_triage],
)


def look_up_item(search_query):
    """Use to find item ID.
    Search query can be a description or keywords."""
    item_id = "item_132612938"
    print("Found item:", item_id)
    return item_id


def execute_refund(item_id, reason="not provided"):
    print("\n\n=== Refund Summary ===")
    print(f"Item ID: {item_id}")
    print(f"Reason: {reason}")
    print("=================\n")
    print("Refund execution successful!")
    return "success"


issues_and_repairs_agent = Agent(
    name="Issues and Repairs Agent",
    instructions=(
        "You are a customer support agent for ACME Inc."
        "Always answer in a sentence or less."
        "Follow the following routine with the user:"
        "1. First, ask probing questions and understand the user's problem deeper.\n"
        " - unless the user has already provided a reason.\n"
        "2. Propose a fix (make one up).\n"
        "3. ONLY if not satesfied, offer a refund.\n"
        "4. If accepted, search for the ID and then execute refund."
        ""
    ),
    tools=[execute_refund, look_up_item, transfer_back_to_triage],
)

Finally, we can run this in a loop (this won't run in python notebooks, so you can try this in a separate python file):

最终，我们可以在循环中运行这个代码（这个代码不能在 Python notebooks 中运行，因此你可以在一个单独的 Python 文件中尝试）:

In [20]:
agent = triage_agent
messages = []

while True:
    user = input("User: ")
    if user == "exit":
        break
    print("User:", user)
    messages.append({"role": "user", "content": user})

    response = run_full_turn(agent, messages)
    agent = response.agent
    messages.extend(response.messages)

User: 你好，我要买一个捉鸟的
Triage Agent: transfer_to_sales_agent({})
Sales Agent: 在抓到鸸鹋方面，你有没有遇到什么问题呢？
User: 我要抓老鹰
Sales Agent: 有趣！但是如果你说的是鸸鹋，ACME有一种超强的“鸸鹋诱惑器”，可以帮助你轻松吸引它们！
User: 是的
Sales Agent: execute_order({'price': 9999, 'product': '鸸鹋诱惑器'})


=== Order Summary ===
Product: 鸸鹋诱惑器
Price: $9999

Order cancelled!
Sales Agent: 价格是9999美元，你还愿意购买吗？
User: 太贵了哥们，9美元 我可以考虑下
Sales Agent: 抱歉，我们的鸸鹋诱惑器固定在9999美元，而且它有一个疯狂的附加条款：购买后，你需要每天对它进行一次唱歌的仪式！你是否愿意继续？
User: 那不要了
Sales Agent: 没问题，如果有其他需求，随时告诉我！
User: ok
Sales Agent: 如有任何问题，请随时告诉我！
User: exirt
Sales Agent: transfer_back_to_triage({})
Triage Agent: 谢谢你的时间！如果需要任何帮助，随时再来找我！


# Swarm

As a proof of concept, we've packaged these ideas into a sample library called [Swarm](https://github.com/openai/swarm). It is meant as an example only, and should not be directly used in production. However, feel free to take the ideas and code to build your own!

作为概念验证，我们已将这些想法打包成一个名为 [Swarm](https://github.com/openai/swarm) 的示例库。它仅作为一个示例，不应直接用于生产环境。不过，欢迎你使用这些想法和代码来构建自己的项目！